In [1]:
import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import accuracy_score

In [2]:
h2o.init()
#df = h2o.import_file("../input/real-quad-fields-1-2/real_quad_fields_1_2.csv")
df = h2o.import_file("C:/Temp/real_quad_fields_1_2.csv")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 14.0.2+12-46, mixed mode, sharing)
  Starting server from C:\Development\Anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\esultano\AppData\Local\Temp\tmphw6oiy2o
  JVM stdout: C:\Users\esultano\AppData\Local\Temp\tmphw6oiy2o\h2o_esultano_started_from_python.out
  JVM stderr: C:\Users\esultano\AppData\Local\Temp\tmphw6oiy2o\h2o_esultano_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_esultano_q2gkjz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.953 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
print(df['class_number'].table())

class_number,Count
1,177159
2,183432


In [37]:
#df = df.split_frame(ratios=[0.25])[0]
#print(df['class_number'].table())

class_number,Count
1,228
2,249


In [4]:
# H2O uses cross validation internally during training (no validation_frame is needed)
train, test = df.split_frame(ratios=[0.3], seed=42)

# Split the data into Train/Validation/Test with Train having 30% and test and validation 35% each
#train, valid, test = df.split_frame(ratios=[.3, .35])

Columns are:
*  'defining_polynomial'
*  'class_number'
*  'discriminant'
*  'res_s=0'
*  'number_roots_unity'
*  'regulator'
*  'a_1', 'a_2', 'a_3', ..., 'a_1000'

In [5]:
primes_up_to_1000 = [1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
x = ["a_" + str(i) for i in primes_up_to_1000]
y = "class_number"

# for binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [6]:
# maybe include "DRF" "GLM" "GBM" "DeepLearning" "XGBoost"
autoML = H2OAutoML(max_runtime_secs=18000, include_algos=["GLM"], balance_classes=True)
autoML.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


See [this stack overflow post](https://stackoverflow.com/questions/55081358/how-to-find-best-params-of-leader-model-in-automl-h2o-python) for details on finding (tuned) hyperparameters of the leader model

In [7]:
autoML.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid__3_AutoML_20210820_232623_model_18,0.659264,0.647992,0.68445,0.499786,0.477682,0.22818
DeepLearning_grid__2_AutoML_20210820_232623_model_2,0.658087,0.656573,0.699974,0.49978,0.482099,0.232419
DeepLearning_grid__1_AutoML_20210820_232623_model_24,0.657337,0.657847,0.684942,0.499913,0.481694,0.232029
DeepLearning_grid__1_AutoML_20210820_232623_model_12,0.653707,0.655026,0.683709,0.499825,0.481172,0.231527
DeepLearning_grid__1_AutoML_20210820_232623_model_23,0.650728,0.673871,0.649852,0.5,0.487527,0.237683
DeepLearning_grid__1_AutoML_20210820_232623_model_19,0.649416,0.666654,0.645919,0.499836,0.486094,0.236288
DeepLearning_grid__2_AutoML_20210820_232623_model_18,0.649138,0.664977,0.669501,0.499874,0.485889,0.236088
DeepLearning_grid__3_AutoML_20210820_232623_model_11,0.645306,0.659991,0.66622,0.495775,0.483581,0.233851
DeepLearning_grid__2_AutoML_20210820_232623_model_8,0.64421,0.667639,0.664297,0.499612,0.487203,0.237366
DeepLearning_grid__3_AutoML_20210820_232623_model_20,0.643983,0.660724,0.666229,0.49966,0.484014,0.234269


In [8]:
m = h2o.get_model("GBM_1_AutoML_20210821_063710")
for key in m.params.keys():
    print("{0}: {1}".format(key, m.params[key]))

dict_keys(['model_id', 'training_frame', 'validation_frame', 'nfolds', 'keep_cross_validation_models', 'keep_cross_validation_predictions', 'keep_cross_validation_fold_assignment', 'fold_assignment', 'fold_column', 'response_column', 'ignored_columns', 'ignore_const_cols', 'score_each_iteration', 'weights_column', 'offset_column', 'balance_classes', 'class_sampling_factors', 'max_after_balance_size', 'max_confusion_matrix_size', 'checkpoint', 'pretrained_autoencoder', 'overwrite_with_best_model', 'use_all_factor_levels', 'standardize', 'activation', 'hidden', 'epochs', 'train_samples_per_iteration', 'target_ratio_comm_to_comp', 'seed', 'adaptive_rate', 'rho', 'epsilon', 'rate', 'rate_annealing', 'rate_decay', 'momentum_start', 'momentum_ramp', 'momentum_stable', 'nesterov_accelerated_gradient', 'input_dropout_ratio', 'hidden_dropout_ratios', 'l1', 'l2', 'max_w2', 'initial_weight_distribution', 'initial_weight_scale', 'initial_weights', 'initial_biases', 'loss', 'distribution', 'quantil

{'default': 0, 'actual': 5, 'input': 5}

In [9]:
y_pred = m.predict(test)
print("Test accuracy:", accuracy_score(h2o.as_list(test)[y].tolist(), h2o.as_list(y_pred)["predict"].tolist()))
print(m.model_performance(test).accuracy())
print(m.model_performance(test).F1())

deeplearning prediction progress: |███████████████████████████████████████| 100%
Test accuracy: 0.5228898851640881
[[0.4475494598502491, 0.6406036317798576]]
[[0.13632983716806205, 0.6742234498563721]]
